# Fit Thermo Groups From Thermo Library

This script takes in user-specified thermo libraries and automatically identifies missing groups, generates group structures, and calculates missing group values using Ridge regression with one-fold cross-validation for hyperparameter tuning. It identifies missing group by checking if the node data is `None`, all zeros, or a string.

This script generates a `group.py` file and replaces the old one.

In [1]:
from rmgpy import settings
from rmgpy.data.thermo import ThermoDatabase, ThermoData, remove_thermo_data, add_thermo_data
import rmgpy.molecule.group as gr
from rmgpy.molecule.group import Group, GroupAtom, GroupBond
from rmgpy.data.base import Entry
from rmgpy.data.base import LogicOr
from rmgpy.molecule.atomtype import ATOMTYPES

import logging
import os
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from copy import copy, deepcopy
import itertools
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use("seaborn-poster")
import numpy as np

from copy import deepcopy
from sklearn.linear_model import RidgeCV,LassoCV,ElasticNetCV,LinearRegression
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error

# User inputs

Input thermo libraries to fit thermo groups from.

In [2]:
libraries = ["C1_C2_Fluorine",
            "C1_C3_hydrofluorocarbons_NIST",  #obtained via email by Linteris on 3/10/23
             "NCSU_C2_C8_PFAS", 
             "PFCA_thermo", 
            ]

# 1. Define helper functions

In [3]:
def check_data(label=None,index=None):
    """
    Helper functions for checking outlier species in the parity plot. 
    User can input either the label or the index of a species.
    This function prints out
    - GAE with estimated GAV: group additivity estimation (GAE) with not exact fit node 
                              (node data is string or None)
    - missing groups
    - GAE without estimated GAV: group additivity estimation with only the exact fit node
    - GAE with fitted GAV: GAE with newly fitted GAVs. 
                           Calculated by directly adding the new groups to the GAE without estimated GAV value.
                           Does not descend the tree again.
    - New GAE: group additivity estimation using the new database. Descending from the new tree.
    """
    if index is None:
        index = label_index_dict[label]
    
    entry = all_data["entry"][index]
    display(entry.item)
    print(entry.item.smiles)
    entry_thermo = all_data["entry thermo"][index]
    
    print("Entry")
    print(entry.short_desc)
    print(entry.label)
    print(entry_thermo)
    print('')
    print("GAE with estimated GAV")
    print(all_data["GAE with estimated GAV"][index])
    print('')
    print("missing")
    print(all_data["missing groups"][index])
    print('')
    print('GAE without estimated GAV')
    print(all_data["GAE without estimated GAV"][index])
    print('')
    print("GAE with fitted GAV")
    print(all_data["GAE with fitted GAV"][index])
    print('')
    if "New GAE" in all_data:
        print("New GAE")
        print(all_data["New GAE"][index])
    
def add_thermo_data_uncertainty(thermo_data1, thermo_data2, group_additivity=True, verbose=False):
    """
    Adapted from the add_thermo_data function from `rmgpy.data.thermo`.
    Add the thermodynamic data, including the uncertainties, from `thermo_data2` to the data `thermo_data1`,
    and return `thermo_data1`.

    If `group_additivity` is True, append comments related to group additivity estimation
    If `verbose` is False, omit the comments from a "zero entry", whose H298, S298, and Cp are all 0.
    If `verbose` is True, or thermo_data2 is not a zero entry, add thermo_data2.comment to thermo_data1.comment.
    """
    if (len(thermo_data1.Tdata.value_si) != len(thermo_data2.Tdata.value_si) or
            any([T1 != T2 for T1, T2 in zip(thermo_data1.Tdata.value_si, thermo_data2.Tdata.value_si)])):
        raise ValueError('Cannot add these ThermoData objects due to their having different temperature points.')

    for i in range(thermo_data1.Tdata.value_si.shape[0]):
        thermo_data1.Cpdata.value_si[i] += thermo_data2.Cpdata.value_si[i]
        thermo_data1.Cpdata.uncertainty_si[i] += thermo_data2.Cpdata.uncertainty_si[i]
    thermo_data1.H298.value_si += thermo_data2.H298.value_si
    thermo_data1.H298.uncertainty_si += thermo_data2.H298.uncertainty_si
    thermo_data1.S298.value_si += thermo_data2.S298.value_si
    thermo_data1.S298.uncertainty_si += thermo_data2.S298.uncertainty_si

    test_zero = sum(abs(value) for value in
                    [thermo_data2.H298.value_si, thermo_data2.S298.value_si] + thermo_data2.Cpdata.value_si.tolist())
    # Used to check if all of the entries in thermo_data2 are zero

    if group_additivity:
        if verbose or test_zero != 0:
            if thermo_data1.comment:
                thermo_data1.comment += ' + {0}'.format(thermo_data2.comment)
            else:
                thermo_data1.comment = 'Thermo group additivity estimation: ' + thermo_data2.comment

    return thermo_data1

def get_neighbors(atom, group_atoms, n_degree_neighbor=1, lone_pair_flag=False):
    """
    Get neighbors within n degree for the center atom recursively.
    
    Args:
        atom (Atom): the center atom
        group_atoms (dict): an empty dictionary
        n_degree_neighbor (int): If a neighbor is n bond apart from the center atom, 
                                    it's defined as n degree neighbor.
        
    
    Returns:
        group_atoms (dict): a dictionary that has Atom object as key and the corresponding 
                            GroupAtom object as value
    """
    
    if n_degree_neighbor == 1:
    
        for atm in atom.edges:
            if atm not in group_atoms:
                if lone_pair_flag: 
                    group_atoms[atm] = GroupAtom(atomtype=[atm.atomtype],
                                             radical_electrons=[atm.radical_electrons],
                                             lone_pairs=[atm.lone_pairs],
                                             label='')
                else: 
                    group_atoms[atm] = GroupAtom(atomtype=[atm.atomtype],
                                             radical_electrons=[atm.radical_electrons],
                                             label='')
    else:
        
        for atm in atom.edges:
            if atm not in group_atoms:
                if lone_pair_flag: 
                    group_atoms[atm] = GroupAtom(atomtype=[atm.atomtype],
                                             radical_electrons=[atm.radical_electrons],
                                             lone_pairs=[atm.lone_pairs],
                                             label='')
                else: 
                    group_atoms[atm] = GroupAtom(atomtype=[atm.atomtype],
                                             radical_electrons=[atm.radical_electrons],
                                             label='')
            get_neighbors(atm, group_atoms, n_degree_neighbor-1)
                
    return group_atoms
        
def make_bonds(atom, group, group_atoms, n_degree_neighbor=1):
    """
    Make bonds between the group atoms in the group object recursively, using the edges of the center atom
    Arg:
        atom (Atom): the center atom for the group
        group (Group): the group object where GroupAtoms are filled but GroupBonds are not added yet
        group_atoms (dict): a dictionary that has Atom object as key and the corresponding 
                            GroupAtom object as value
        n_degree_neighbor (int): If a neighbor is n bond apart from the center atom, 
                                    it's defined as n degree neighbor.
    Returns:
        group (Group): the group object with GroupBonds added
    """
    
    if n_degree_neighbor == 1:
        
        for bonded_atom, bond in atom.edges.items():
            if not group.has_bond(group_atoms[atom],group_atoms[bonded_atom]):
                group.add_bond(GroupBond(group_atoms[atom],group_atoms[bonded_atom],order=[bond.order]))
            else:
                pass
                
    else:
        
        for bonded_atom, bond in atom.edges.items():
            if not group.has_bond(group_atoms[atom],group_atoms[bonded_atom]):
                group.add_bond(GroupBond(group_atoms[atom],group_atoms[bonded_atom],order=[bond.order]))
            else:
                pass
            make_bonds(bonded_atom, group, group_atoms, n_degree_neighbor-1)
            
    return group

def make_group(atom, parent_node, n_degree_neighbor=1):
    """
    Make a group with `atom` as the center atom and `n_degree_neighbor` degree of neighbor, 
    using `get_neighbors` and `make_bonds` functions.
    
    Args:
        atom (Atom): the center atom for the group
        n_degree_neighbor (Int): If a neighbor is n bond apart from the center atom, 
                                    it's defined as n degree neighbor.
    Returns:
        group (Group): the group containing the center atom and the n degrees of neighbor
    """

    group_atoms = {}
    bonds = []
        
    #let's look at the specifics of the atoms on the parent node
    for atm in parent_node.item.atoms:  
        if atm.is_fluorine: 
            if atm.lone_pairs==[]:  # if the parent node does not specify lone pairs, then the child shouldn't have to
                lone_pair_flag = False #this flag will tell us NOT to make the child group more specific
            else: 
                lone_pair_flag = True #but if the parent group is specific, make the child group specific 
        
    group_atoms = get_neighbors(atom, group_atoms, n_degree_neighbor=n_degree_neighbor, lone_pair_flag=lone_pair_flag) #this is a dictionary
    # print(group_atoms)
    
    #now let's look at each of the group atoms 
    

    if atom.atomtype.label in ["O2s", "S2s", "O4tc", "Cs"]:
        group_atoms[atom] = GroupAtom(atomtype=[atom.atomtype],
                                     radical_electrons=[atom.radical_electrons],
                                     label='*')
    else:
        group_atoms[atom] = GroupAtom(atomtype=[atom.atomtype],
                                     radical_electrons=[atom.radical_electrons],
                                     lone_pairs=[atom.lone_pairs],
                                     charge=[atom.charge],
                                     label='*')
    
    group = Group(atoms=list(group_atoms.values()))
    
    group = make_bonds(atom, group, group_atoms, n_degree_neighbor=n_degree_neighbor)

    group.update()

    return group    

def make_neighbor_name(atom,n_degree_neighbor=1,exclude=[]):
    """
    Make the name for n degree of neighbor recursively
    Args:
        atom (Atom): the center atom for the group
        n_degree_neighbor (Int): If a neighbor is n bond apart from the center atom, 
                                    it's defined as n degree neighbor.
        exclude (list of Atom): helper list to record atoms that have been named to avoid repetition 
        
    Returns:
        neighbors (string): name for n degrees of neighbors
        exclude (list of Atom): helper list to record atoms that have been named to avoid repetition 
        
    """
    
    if n_degree_neighbor == 1:

        names = []

        for atom2 in atom.edges.keys():
            
            if atom2.atomtype.label != 'H':
                
                if atom2 not in exclude:
                    atom_neighbor = atom2.atomtype.label
                    names.append(atom_neighbor)
                    exclude.append(atom2)
                
        neighbors = ''.join(sorted(names))
        neighbors += 'H' * len(['H' for atom2 in atom.edges.keys() if atom2.atomtype.label == 'H' and atom2 not in exclude])
        
        return neighbors, exclude

        
    else:
        
        names = []
        
        if atom not in exclude:
            exclude.append(atom)
        
        for atom2 in atom.edges.keys():
            
            if atom2.atomtype.label != 'H':
                
                atom_neighbor = ''
                
                if atom2 not in exclude:
                    
                    atom_neighbor += atom2.atomtype.label
                    exclude.append(atom2)
                    
                names.append(atom_neighbor)
                    
        for (i,atom2) in enumerate(atom.edges.keys()):
                
            atom2_neighbor, exclude = make_neighbor_name(atom2, n_degree_neighbor-1,exclude=exclude)

            if atom2_neighbor:
                names.append(f"({atom2_neighbor})")
                
        neighbors = ''.join(sorted(names))
        neighbors += 'H' * len(['H' for atom2 in atom.edges.keys() if atom2.atomtype.label == 'H' and atom2 not in exclude])
        
        for atom2 in atom.edges.keys():
            if atom2 not in exclude:
                exclude.append(atom2)
        
        return neighbors, exclude
    
def make_group_name(atom,n_degree_neighbor=1):
    """
    Make the group string for a group with atom as its center atom and n degree of neighbor, 
    using `make_neighbor_name`
    Args:
        atom (Atom): the center atom for the group
        n_degree_neighbor (Int): If a neighbor is n bond apart from the center atom, 
                                    it's defined as n degree neighbor.
        
    Returns:
        group_str (string): name for the group
    """
    
    
    exclude = []
    group_str = atom.atomtype.label
    
    neighbors = make_neighbor_name(atom,n_degree_neighbor=n_degree_neighbor,exclude=exclude)[0]
    
    if atom.radical_electrons==1:
        group_str += "J"

    if neighbors:
        group_str += f'-{neighbors}'
        
    return group_str
    


## 1.1 Special groups that we should not fit

`special_list` contains groups that always appear with other groups such that these groups are not independent. For example, `O2d-CO` always appear with `CO-O2d`, and `S2d-CS` always appear with `CS-S2d`. In the past, RMG developers chose to set these groups to be zeros, so we don't generate new groups on them. This list needs to be updated when the relevant groups are changed.

In [4]:
special_list = [
    'O0sc-N5dc',
    'S2d-CS',
    'S2d-C2d',
    'O2d-N5dc',
    'O2d-CO',
    'O2d-S6dd',
    'O2d-N3d',
    'O2d-S4dd',
    'O2d-S4d',
    'O2d-Cdd',
    'Cb-CSCbCb',
    'Cdd-CdO2d',
]

# 2 Fit thermo groups

## 2.1 Load database

In [5]:
database = ThermoDatabase()
database.load(os.path.join(settings['database.directory'],"thermo"),
             libraries = libraries,
             depository=False)

## Debug!! 

In [ ]:
#let's see if there are lone_pairs in the parent node
#check for specifics
node0.item.atoms[0].lone_pairs
#lone_pairs
#charge

In [ ]:
molecule.atoms[0].lone_pairs

In [ ]:
for atom in molecule.atoms:
    print(atom.atomtype)

In [ ]:
#here is our problem. CHF is being loaded in with these atomtypes. Why?
print(atom.atomtype)
for atm in atom.edges:
    print(atm.atomtype)
    
#this is happens as soon as the molecule is loaded
for atom in molecule.atoms:
    print(atom.atomtype)

In [ ]:
#this is the parent node that is being matched. 


In [ ]:
#this is the adjacency list of the parent node
print(node.item.to_adjacency_list())

In [ ]:
#these are the children for the node that currently exist

In [ ]:
node.children[0].item

In [ ]:
print(node.children[0].item.to_adjacency_list())

In [ ]:
node.children[1].item

In [ ]:
print(node.children[1].item.to_adjacency_list())

In [ ]:
node.children[2].item

In [ ]:
print(node.children[2].item.to_adjacency_list())

In [ ]:
#This is the first attempt to make this a group

In [ ]:
group_0.atoms[1].lone_pairs

In [ ]:
print(group_0.to_adjacency_list())
#I think the problem lies in the fact that these are being made as a C2s and F1s

In [ ]:
#This is when it cycles through "make_group"
group

In [ ]:
print(group.to_adjacency_list())


In [ ]:
print(molecule.to_adjacency_list())

In [ ]:
dir(x.data)

In [ ]:
node0.long_desc


## 2.2 Get entry data from thermo libraries, identify missing groups, generate group structure for missing groups, obtain old GA estimation

In [6]:
all_data = dict()
all_data["entry"] = list()
all_data["entry thermo"] = list()
all_data["GAE without estimated GAV"] = list()
all_data["GAE with estimated GAV"] = list()
all_data["missing GAE"] = list()
all_data["missing groups"] = list()

missing_group_index_dict = dict()
missing_group_dict = dict()
missing_group_index = 0

not_missing = []

for library in database.libraries:
    
    entries=list(database.libraries[library].entries.items())
    
    for item, entry in entries:
        
        if entry.data is not None:
            
            if not isinstance(entry.data, ThermoData):
                try:
                    entry_thermo = entry.data.to_thermo_data()
                except:
                    continue
            else:
                entry_thermo = entry.data

            molecule = entry.item
            print("\n========================================================")
            print(f"Current species is {entry} ({molecule.get_formula()})\n")
            
            if molecule.smiles in ["[Ar]","[He]","[Ne]"]:
                #Current group additivity tree only contains C, N, S, O, and some halogen chemistry
                #Use `primaryThermoLibrary` for noble gas thermo
                continue
            if molecule.is_radical():
                print('GAV not built for radicals.')
            if not molecule.is_radical():
                #Current notebook only works on non-radical species.
                #Should be able to extend for radical groups with careful adaptation
                
                estimated_thermo = database.estimate_thermo_via_group_additivity(molecule)
                
                if (estimated_thermo.H298.value_si-entry_thermo.H298.value_si)/4180 < 2:
                    print('Estimated thermo is already good')
                    molecule.sort_atoms()
                    for atom in molecule.atoms:
                        if atom.is_non_hydrogen() and not atom.is_halogen():
                            node0 = database.groups['group'].descend_tree(molecule, {'*': atom}, None) #this is the "parent" node
                            data = node0.data
                            print(f"The node used to estimate the GAV for this molecule was {node0}. Good estimation.")
                            print(f"The long desc is: {node0.long_desc}")

                if (estimated_thermo.H298.value_si-entry_thermo.H298.value_si)/4180 > 2:
                    #Don't bother to fit new groups if the `estimated_thermo` is already good
                    
                    print('Estimated thermo is off.')
                    
                    missing_grp = list()
                    missing = 0

                    real_data_thermo = ThermoData(
                            Tdata=([300, 400, 500, 600, 800, 1000, 1500], "K"),
                            Cpdata=([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "J/(mol*K)"),
                            H298=(0.0, "kJ/mol"),
                            S298=(0.0, "J/(mol*K)"),
                        )

                    molecule.sort_atoms()
                    
                    for atom in molecule.atoms:
                        #here we loop through each atom. If atom is hydrogen or halogen, we don't descend
                        #it down tree because we don't consider hydrogen or halogen as center atom.
                        #If the node data is None, string, or all zeros, we go in the branch of generating
                        #missing group structure. If the node data is just normal data, we add it to
                        #`real_data_thermo`.

                        if atom.is_non_hydrogen() and not atom.is_halogen():
                            #Hydrogen and halogen are not considered as center atom

                            node0 = database.groups['group'].descend_tree(molecule, {'*': atom}, None) #this is the "parent" node
                            node = node0
                            data = node.data

                            add_to_real_data_thermo = True
                            print(f"The parent node is: {node}")
                            
                            ###############################################################################
                                #Start identifying missing group
                            if data is None or isinstance(data,str) or data.is_all_zeros():
                                print('missing group')
                                #make group structure and group string
                                n_degree_neighbor = 1
                                
                                #let's pass in the parent node to make_group
                                parent_node = node0
                                print(f'Parent node might need retraining: {parent_node.long_desc}')
                                print(f'Parent data is {parent_node.data}')
                                group = make_group(atom, parent_node, n_degree_neighbor=n_degree_neighbor)
                                group_str = make_group_name(atom, n_degree_neighbor=n_degree_neighbor)
                                group_0 = group
                                group_str_0 = group_str
                                print(f"First initial generated group is: {group_str_0}")
                                print(group_0.to_adjacency_list())
                                
                                if group_str.split("_")[-1] not in special_list:
                                    
                                    # print('starting while loop one')
                                    while not group.is_subgraph_isomorphic(node.item):
                                    #while not group.is_subgraph_isomorphic(node.item, generate_initial_map = True):#Su suggested this
                                        #new group has to be the child-node of the originally matched group
                                        #to have correct parent-node child-node relation
                                        # print(f"newly made group \n{group.to_adjacency_list()} is not subgraph isomorphic to the parent node {node} \n{node.item.to_adjacency_list()}")
                                        n_degree_neighbor+=1
                                        group = make_group(atom, n_degree_neighbor=n_degree_neighbor)
                                        # print('made group')
                                        group_str = make_group_name(atom, n_degree_neighbor=n_degree_neighbor)
                                        # print('made group name')
                                        # print(n_degree_neighbor)
                                        # assert n_degree_neighbor < 20
                                    # print('starting while loop 2')
                                    while any([(group.make_sample_molecule()).is_subgraph_isomorphic(child.item, generate_initial_map = True) for child in node.children]):
                                        #Child-node can't be the child of other children
                                        #to avoid ambiguous group selection
                                        n_degree_neighbor+=1
                                        group = make_group(atom, n_degree_neighbor=n_degree_neighbor)
                                        group_str = make_group_name(atom, n_degree_neighbor=n_degree_neighbor)
                                    # print('exited while loops')
                                if group_str.split("_")[-1] not in special_list:
                                    add_to_real_data_thermo = False

                                    group.sort_atoms()

                                    group_str = f'{node.label}_{group_str}'

                                    missing += 1
                                    missing_grp.append(group_str)
                                    if group_str not in missing_group_index_dict:
                                        missing_group_index_dict[group_str] = missing_group_index
                                        missing_group_index+=1

                                        missing_group_dict[group_str] = dict()
                                        missing_group_dict[group_str]["group"] = [group]
                                        missing_group_dict[group_str]["atom"] = [atom]
                                        missing_group_dict[group_str]["molecule"] = [molecule]
                                        missing_group_dict[group_str]["label"] = [entry.label]
                                    else:
                                        missing_group_dict[group_str]["group"].append(group)
                                        missing_group_dict[group_str]["atom"].append(atom)
                                        missing_group_dict[group_str]["molecule"].append(molecule)
                                        missing_group_dict[group_str]["label"].append(entry.label)

                ###############################################################################
                #calculate real data thermo value
                            if add_to_real_data_thermo:
                                while node is not None and node.data is None:
                                    node = node.parent
                                if node is None:
                                    raise DatabaseError(f'Unable to determine thermo parameters for atom {atom} in molecule {molecule}: '
                                                        f'no data for node {node0} or any of its ancestors in database {database.label}.')

                                data = node.data
                                comment = node.label
                                loop_count = 0
                                while isinstance(data, str):
                                    loop_count += 1
                                    if loop_count > 100:
                                        raise DatabaseError("Maximum iterations reached while following thermo group data pointers. A circular"
                                                            f" reference may exist. Last node was {node.label} pointing to group called {data} in "
                                                            f"database {database.label}")

                                    for entr in database.groups["group"].entries.values():
                                        if entr.label == data:
                                            data = entr.data
                                            comment = entr.label
                                            break
                                    else:
                                        raise DatabaseError(f"Node {node.label} points to a non-existing group called {data} "
                                                            f"in database {database.label}")

                                data.comment = '{0}({1})'.format(database.groups['group'].label, comment)
                                add_thermo_data(real_data_thermo, data, group_additivity=True)

                    cyclic = molecule.is_cyclic()

                    if cyclic:
                        sssr = molecule.get_smallest_set_of_smallest_rings()
                        for ring in sssr:
                            for atomPair in itertools.permutations(ring, 2):
                                try:
                                    database._add_group_thermo_data(real_data_thermo, database.groups['longDistanceInteraction_cyclic'], molecule,
                                                                {'*1': atomPair[0], '*2': atomPair[1]})
                                except KeyError:
                                    pass

                    # Do ring corrections separately because we only want to match
                    # each ring one time

                    if cyclic:
                        monorings, polyrings = molecule.get_disparate_cycles()
                        for ring in monorings:
                            # Make a temporary structure containing only the atoms in the ring
                            # NB. if any of the ring corrections depend on ligands not in the ring, they will not be found!
                            try:
                                database._add_ring_correction_thermo_data_from_tree(real_data_thermo, database.groups['ring'], molecule, ring)
                            except KeyError:
                                logging.error("Couldn't find a match in the monocyclic ring database even though "
                                              "monocyclic rings were found.")
                                logging.error(molecule)
                                logging.error(molecule.to_adjacency_list())
                                raise
                        for polyring in polyrings:
                            # Make a temporary structure containing only the atoms in the ring
                            # NB. if any of the ring corrections depend on ligands not in the ring, they will not be found!
                            try:
                                database._add_polycyclic_correction_thermo_data(real_data_thermo, molecule, polyring)
                            except KeyError:
                                logging.error("Couldn't find a match in the polycyclic ring database even though "
                                              "polycyclic rings were found.")
                                logging.error(molecule)
                                logging.error(molecule.to_adjacency_list())

    ########################################################################################################
                    if missing > 0:
            #If there are missing groups identified in this molecule, add to all_data (dict)
                        entry.short_desc = library
                        all_data["entry"].append(entry)
                        all_data["GAE without estimated GAV"].append(real_data_thermo)
                        all_data["GAE with estimated GAV"].append(estimated_thermo)
                        all_data["missing groups"].append(missing_grp)
                        all_data["entry thermo"].append(entry_thermo)

                        try:
                            #we remove the contribution from `real_data_thermo` in the old thermo estimation
                            #the rest of thermo are contributed by the missing groups
                            missing_group_thermo = remove_thermo_data(deepcopy(entry_thermo),real_data_thermo)
                            all_data["missing GAE"].append(missing_group_thermo)
                        except (ValueError,IndexError):
                            #We need Cp0 and CpInf to perform the inversion from Nasa to ThermoData
                            if entry_thermo.Cp0 is None:
                                cp_0 = molecule.calculate_cp0()
                                entry_thermo.Cp0 = (cp_0, "J/(mol*K)")
                            if entry_thermo.CpInf is None:
                                cp_inf = molecule.calculate_cpinf()
                                entry_thermo.CpInf = (cp_inf, "J/(mol*K)")

                            nasa = entry_thermo.to_nasa(Tmin=10.0, Tmax=3000.0, Tint=500.0)
                            entry_thermo = nasa.to_thermo_data()
                            missing_group_thermo = remove_thermo_data(deepcopy(entry_thermo),real_data_thermo)
                            all_data["missing GAE"].append(missing_group_thermo)
                    if missing == 0:
                        #if there are no missing groups, let's isolate these species and do something with them later
                        print('no groups missing')
        # print('completed')
spc_num = len(all_data["entry"])
grp_num = len(missing_group_index_dict.keys())
print(f"Fitting {grp_num} of new groups with {spc_num} of species")


Current species is F (F)

GAV not built for radicals.

Current species is F2 (F2)

Estimated thermo is already good

Current species is HF (FH)

Estimated thermo is off.
no groups missing

Current species is CF (CF)

GAV not built for radicals.

Current species is CHF (CHF)

Estimated thermo is already good
The node used to estimate the GAV for this molecule was CJ2_singlet-F. Good estimation.
The long desc is: 

Current species is CHF(T) (CHF)

GAV not built for radicals.

Current species is CH2F (CH2F)

GAV not built for radicals.

Current species is CH3F (CH3F)

Estimated thermo is off.
The parent node is: CsFHHH
missing group
Parent node might need retraining: Fitted using sklearn Ridge regression with alpha = 1e-06
Library | Number of Species
CHOF_G4 |         1
Parent data is ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([0,0,0,0,0,0,0],'J/(mol*K)'), H298=(0,'kJ/mol'), S298=(0,'J/(mol*K)'), comment="""group(CsFHHH)""")
First initial generated group is: Cs-F1sHH

TypeError: make_group() missing 1 required positional argument: 'parent_node'

## 2.3 Construct A and b

In [ ]:
group_data_items = [("H298","J/mol"), ("S298","J/(mol*K)"), 
                    ("Cp300","J/(mol*K)"), ("Cp400","J/(mol*K)"), 
                    ("Cp500","J/(mol*K)"), ("Cp600","J/(mol*K)"), 
                    ("Cp800","J/(mol*K)"), ("Cp1000","J/(mol*K)"), 
                    ("Cp1500","J/(mol*K)")]

#Initialize A and b
A = np.zeros((spc_num,grp_num))
b = np.zeros((spc_num,len(group_data_items)))

#Fill in A and b
label_index_dict=dict()
for spc_index, entry in enumerate(all_data["entry"]):
    label_index_dict[entry.label]=spc_index
    groups = all_data["missing groups"][spc_index]
    for group in groups:
        A[spc_index, missing_group_index_dict[group]]+=1
    
    thermo_data = all_data["missing GAE"][spc_index]
    b[spc_index,0] = thermo_data.H298.value_si
    b[spc_index,1] = thermo_data.S298.value_si
    b[spc_index,2:9] = thermo_data.Cpdata.value_si

## 2.4 Perform ridge regression using one-fold cross-validation hyperparameter tuning

In [ ]:
#Ridge regression using one-fold cross-validation hyperparameter tuning
soln_dict = dict()
soln = RidgeCV(alphas=[1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1],fit_intercept=False,normalize=False,store_cv_values=True).fit(A,b)
soln_dict["RidgeCV"]=soln
alpha = soln_dict["RidgeCV"].alpha_
print(f"The optimal hyperparameter is {alpha}, found by one-fold cross-validation.")

### 2.4.1 Check RMSE as a function of hyperparameter (alpha)

In [ ]:
for ind,item in enumerate(group_data_items):
    plt.figure()
    cv = soln_dict["RidgeCV"].cv_values_
    rmse = [sum((cv[:,ind,alpha_ind])**0.5)/cv.shape[0] for alpha_ind in range(cv.shape[2])]
    plt.plot(soln_dict["RidgeCV"].alphas,rmse)
    plt.title(f"RMSE of {item[0]} ({item[1]})")
    plt.xscale("log")

## 2.5 Calculate uncertainty

In [ ]:
b_pred = soln_dict["RidgeCV"].predict(A)

standard_error = dict()
for i in range(b.shape[1]):
    residuals = b[:,i] - b_pred[:,i]
    sigma_squared_hat = residuals.T @ residuals/(A.shape[0] - A.shape[1])
    #See https://arxiv.org/abs/1509.09169 Section 1.4.2 for derivation
    var_beta_hat = sigma_squared_hat * np.linalg.inv(A.T @ A + alpha*np.identity(A.shape[1])) @ A.T @ A @ (np.linalg.inv(A.T @ A + alpha*np.identity(A.shape[1]))).T
    standard_error[i] = np.diagonal(var_beta_hat)**0.5

## 2.6 Write the fitted data to ThermoData format

In [ ]:
fitted_group_data = dict()
x = soln_dict["RidgeCV"].coef_.T

for i, label in enumerate(missing_group_index_dict.keys()):
    if all(x[i,:]==0.0):
        group_label = label.split("_")[1]
        print(f"All fitted item for group {group_label} from species index {i} are zeros. {group_label} is a special group. Add {group_label} to special_list")
    H298, S298, Cpdata = x[i,0], x[i,1], x[i,2:9]
    H298_unc, S298_unc, Cpdata_unc = standard_error[0][i], standard_error[1][i], [standard_error[j][i] for j in range(2,9)]
    fitted_group_data[label] = ThermoData(
            Tdata=([300, 400, 500, 600, 800, 1000, 1500], "K"),
            Cpdata=(Cpdata, "J/(mol*K)", "+|-", Cpdata_unc),
            H298=(H298/1000, "kJ/mol", "+|-", H298_unc/1000),
            S298=(S298, "J/(mol*K)", "+|-", S298_unc),
            comment=f"fitted({label})",
        )

## 2.7 Calculate GA esitmated thermo using newly fitted groups

In [ ]:
all_data["GAE with fitted GAV"]=list()

for spc_index, groups in enumerate(all_data["missing groups"]):
    
    GAE_without_estimated_GA = all_data["GAE without estimated GAV"][spc_index]
    
    fitted_thermo = deepcopy(GAE_without_estimated_GA)
    for group in groups:
        add_thermo_data_uncertainty(fitted_thermo,fitted_group_data[group])
        
    all_data["GAE with fitted GAV"].append(fitted_thermo)


# 3. Check quality of new groups

## 3.1 Look at parity plot for training error and uncertainty from fitting

In [ ]:
colormap = mpl.cm.tab20
plt.figure(figsize=(10,10))
entry_values = list()
old_GAE_values = list()
new_GAE_values = list()
new_GAE_uncertainties = list()
new_new_GAE_values = list()

for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_values.append(entry_thermo.H298.value_si/4180)
    estimated_GAE = all_data["GAE with estimated GAV"][spc_index]
    old_GAE_values.append(estimated_GAE.H298.value_si/4180)
    fitted_GAE = all_data["GAE with fitted GAV"][spc_index]
    new_GAE_values.append(fitted_GAE.H298.value_si/4180)
    new_GAE_uncertainties.append(fitted_GAE.H298.uncertainty_si/4180)
    
    
plt.figure(figsize=(10,10))
plt.errorbar(entry_values,old_GAE_values,fmt='o')
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
RMSE = mean_squared_error(entry_values,old_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,old_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} kcal/mol')
plt.xlabel("H298: Entry data (kcal/mol)")
plt.ylabel("H298: Old estimation (kcal/mol)")

plt.figure(figsize=(10,10))
plt.errorbar(x=entry_values,y=new_GAE_values,yerr=new_GAE_uncertainties,fmt='o')
for i, entry_value in enumerate(entry_values):
    if abs(entry_value - new_GAE_values[i])>15:
        plt.text(entry_value, new_GAE_values[i], s=all_data["entry"][i].label)
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
plt.xlabel("H298: Entry data (kcal/mol)")
plt.ylabel("H298: New estimation with fitted data (kcal/mol)")
RMSE = mean_squared_error(entry_values,new_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,new_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} kcal/mol')

In [ ]:
colormap = mpl.cm.tab20
plt.figure(figsize=(10,10))
entry_values = list()
old_GAE_values = list()
new_GAE_values = list()
new_GAE_uncertainties = list()
new_new_GAE_values = list()


for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_values.append(entry_thermo.S298.value_si/4.180)
    estimated_GAE = all_data["GAE with estimated GAV"][spc_index]
    old_GAE_values.append(estimated_GAE.S298.value_si/4.180)
    fitted_GAE = all_data["GAE with fitted GAV"][spc_index]
    new_GAE_values.append(fitted_GAE.S298.value_si/4.180)
    new_GAE_uncertainties.append(fitted_GAE.S298.uncertainty_si/4.180)

plt.figure(figsize=(10,10))
plt.errorbar(entry_values,old_GAE_values,fmt='o')
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
RMSE = mean_squared_error(entry_values,old_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,old_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} cal/(mol*K)')
plt.xlabel("S298: Entry data (cal/(mol*K))")
plt.ylabel("S298: Old estimation (cal/(mol*K))")

plt.figure(figsize=(10,10))
plt.errorbar(x=entry_values,y=new_GAE_values,yerr=new_GAE_uncertainties,fmt='o')
for i, entry_value in enumerate(entry_values):
    if abs(entry_value - new_GAE_values[i])>5:
        plt.text(entry_value, new_GAE_values[i], s=all_data["entry"][i].label)
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
plt.xlabel("S298: Entry data (cal/(mol*K))")
plt.ylabel("S298: New estimation with fitted data (cal/(mol*K))")
RMSE = mean_squared_error(entry_values,new_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,new_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} cal/(mol*K)')

# plt.figure()
# plt.hist(np.array(entry_values)-np.array(new_GAE_values),alpha=0.5,label="New GA estimation")
# plt.hist(np.array(entry_values)-np.array(old_GAE_values),alpha=0.5,label="Old GA estimation")
# plt.xlabel("S298 error (cal/mol-K)")
# plt.ylabel("Counts (-)")
# plt.legend()

## 3.2 Look at generalizability of the new groups by comparing the old estimation and the one-fold cross-validation of new estimation

In [ ]:
alpha = soln_dict["RidgeCV"].alpha_
all_data["cross validation estimation"]=list()
for skip_spc_index in range(spc_num):

    A = np.zeros((spc_num,grp_num))
    b = np.zeros((spc_num,len(group_data_items)))

    for spc_index, entry in enumerate(all_data["entry"]):
        if spc_index == skip_spc_index:
            pass
        else:
            groups = all_data["missing groups"][spc_index]
            for group in groups:
                A[spc_index, missing_group_index_dict[group]]+=1

            thermo_data = all_data["missing GAE"][spc_index]
            b[spc_index,0] = thermo_data.H298.value_si
            b[spc_index,1] = thermo_data.S298.value_si
            b[spc_index,2:9] = thermo_data.Cpdata.value_si

    soln = Ridge(alpha=alpha,fit_intercept=False,normalize=False, max_iter=1e5).fit(A,b)
    soln_dict["Ridge"]=soln

    b_pred = soln_dict["Ridge"].predict(A)

    standard_error = dict()
    for i in range(b.shape[1]):
        residuals = b[:,i] - b_pred[:,i]
        sigma_squared_hat = residuals.T @ residuals/(A.shape[0] - A.shape[1])
        var_beta_hat = sigma_squared_hat * np.linalg.inv(A.T @ A + alpha*np.identity(A.shape[1])) @ A.T @ A @ (np.linalg.inv(A.T @ A + alpha*np.identity(A.shape[1])))
        standard_error[i] = np.diagonal(var_beta_hat)**0.5

    fitted_group_data_cv = dict()

    for i, label in enumerate(missing_group_index_dict.keys()):
        x = soln_dict["Ridge"].coef_.T
        H298, S298, Cpdata = x[i,0], x[i,1], x[i,2:9]
        H298_unc, S298_unc, Cpdata_unc = standard_error[0][i], standard_error[1][i], [standard_error[j][i] for j in range(2,len(group_data_items))]
        fitted_group_data_cv[label] = ThermoData(
                Tdata=([300, 400, 500, 600, 800, 1000, 1500], "K"),
                Cpdata=(Cpdata, "J/(mol*K)", "+|-", Cpdata_unc),
                H298=(H298/1000, "kJ/mol", "+|-", H298_unc/1000),
                S298=(S298, "J/(mol*K)", "+|-", S298_unc),
                comment=f"fitted({label})",
            )

    groups = all_data["missing groups"][skip_spc_index]

    GAE_without_estimated_GA = all_data["GAE without estimated GAV"][skip_spc_index]

    fitted_thermo = deepcopy(GAE_without_estimated_GA)
    for group in groups:
        add_thermo_data_uncertainty(fitted_thermo,fitted_group_data_cv[group])

    all_data["cross validation estimation"].append(fitted_thermo)

colormap = mpl.cm.tab20
plt.figure(figsize=(10,10))
entry_values = list()
old_GAE_values = list()
cross_validation_values = list()

for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_values.append(entry_thermo.H298.value_si/4180)
    estimated_GAE = all_data["GAE with estimated GAV"][spc_index]
    old_GAE_values.append(estimated_GAE.H298.value_si/4180)
    cross_validation_GAE = all_data["cross validation estimation"][spc_index]
    cross_validation_values.append(cross_validation_GAE.H298.value_si/4180)

plt.figure()
RMSE1 = mean_squared_error(entry_values,cross_validation_values,squared=False)
MAE1 = mean_absolute_error(entry_values,cross_validation_values)
RMSE = mean_squared_error(entry_values,old_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,old_GAE_values)
plt.hist([np.array(entry_values)-np.array(cross_validation_values), np.array(entry_values)-np.array(old_GAE_values)],alpha=0.5,label=[f"Cross validation GA estimation, RMSE: {RMSE1:.1f}, MAE: {MAE1:.1f} kcal/mol",f"Old GA estimation, RMSE: {RMSE:.1f}, MAE: {MAE:.1f} kcal/mol"])
plt.xlabel("H298 error (kcal/mol)")
plt.ylabel("Counts (-)")
plt.legend()

colormap = mpl.cm.tab20
plt.figure(figsize=(10,10))
entry_values = list()
old_GAE_values = list()
cross_validation_values = list()

for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_values.append(entry_thermo.S298.value_si/4.180)
    estimated_GAE = all_data["GAE with estimated GAV"][spc_index]
    old_GAE_values.append(estimated_GAE.S298.value_si/4.180)
    cross_validation_GAE = all_data["cross validation estimation"][spc_index]
    cross_validation_values.append(cross_validation_GAE.S298.value_si/4.180)
    
plt.figure()
RMSE1 = mean_squared_error(entry_values,cross_validation_values,squared=False)
MAE1 = mean_absolute_error(entry_values,cross_validation_values)
RMSE = mean_squared_error(entry_values,old_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,old_GAE_values)
plt.hist([np.array(entry_values)-np.array(cross_validation_values), np.array(entry_values)-np.array(old_GAE_values)],alpha=0.5,label=[f"Cross validation GA estimation, RMSE: {RMSE1:.1f}, MAE: {MAE1:.1f} cal/(mol*K)", f"Old GA estimation, RMSE: {RMSE:.1f}, MAE: {MAE:.1f} cal/(mol*K)"])
plt.xlabel("S298 error (cal/(mol*K))")
plt.ylabel("Counts (-)")
plt.legend()

## 3.2 Check if there's bad fit
This block checks if there are any bad fit groups. User should expect this block prints out nothing. It this block prints out anything, user should not trust the fitted results and should open an issue or try to identify the reason of the poor fit. It uses following criteria to find bad fit:

- Error in H298 and S298 from the new estimation is worse than the old estimation. This is usually caused by special groups. Try identify the special groups and put in the `special_list`, and re-run the code. 

- Error in H298 from the new estimation `> n kcal/mol` or S298 `> n cal/(mol* K)`, where n is default to 5. Increase n at your own risk.

In [ ]:
bad_fit_spc_index_list = list()
n=5
for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_thermo = all_data["entry thermo"][spc_index]
    diff = abs(entry_thermo.H298.value_si-all_data["GAE with estimated GAV"][spc_index].H298.value_si)
    diff_new = abs(entry_thermo.H298.value_si-all_data["GAE with fitted GAV"][spc_index].H298.value_si)
    if diff<diff_new:
        bad_fit_spc_index_list.append(spc_index)
    if diff_new>n*4.18*1000:
        bad_fit_spc_index_list.append(spc_index)
    diff = abs(entry_thermo.S298.value_si-all_data["GAE with estimated GAV"][spc_index].S298.value_si)
    diff_new = abs(entry_thermo.S298.value_si-all_data["GAE with fitted GAV"][spc_index].S298.value_si)
    if diff<diff_new:
        bad_fit_spc_index_list.append(spc_index)
    if diff_new>n*4.18:
        bad_fit_spc_index_list.append(spc_index)
        
bad_fit_spc_index_list = list(set(bad_fit_spc_index_list))
print(f"{len(bad_fit_spc_index_list)} out of {len(entries)} species have bad fits")

In [ ]:
for spc_index in bad_fit_spc_index_list:
    print("Warning: these are bad fits:")
    check_data(index=spc_index)
    print("===========================================================================")

# 4. Write fitted data to database

In [ ]:
database_new = deepcopy(database)
duplicate = dict()

for group_str in missing_group_dict.keys():
    group = missing_group_dict[group_str]["group"][0]
    group_atom = missing_group_dict[group_str]["atom"][0]
    molecule = missing_group_dict[group_str]["molecule"][0]
    entry_label = group_str.split("_")[-1]
    
    #Make sure there are no duplicate entry label
    if entry_label in database_new.groups['group'].entries:
        entry_data = database_new.groups['group'].entries[entry_label].data
        if entry_data is not None and not isinstance(entry_data,str):
            if not database_new.groups['group'].entries[entry_label].data.is_all_zeros():
                counter = 0
                while entry_label in database_new.groups['group'].entries:
                    print(entry_label)
                    if entry_label not in duplicate:
                        duplicate[entry_label] = dict()
                        duplicate[entry_label]["group"] = group
                        duplicate[entry_label]["atom"] = group_atom
                        duplicate[entry_label]["molecule"] = molecule
                    counter += 1
                    entry_label = entry_label.split('_')[0]
                    entry_label += '_{0}'.format(counter)
                    if entry_label not in duplicate:
                        duplicate[entry_label] = dict()
                        duplicate[entry_label]["group"] = group
                        duplicate[entry_label]["atom"] = group_atom
                        duplicate[entry_label]["molecule"] = molecule

    string = ''
    for ind in np.nonzero(A[:,missing_group_index_dict[group_str]])[0]:
        string += f'{all_data["entry"][ind].item.smiles} ({all_data["entry"][ind].label}) from {all_data["entry"][ind].short_desc}\n'
        
    node0 = database_new.groups['group'].descend_tree(molecule, {'*': group_atom}, None)
    
    # make L3 dummy node
    if node0.label in ["C", "O", "N", "S"]:
        entry = Entry(index = len(database_new.groups['group'].entries)+1,
                        label = group_atom.atomtype.label,
                        item = Group().from_adjacency_list(f"""1 * {group_atom.atomtype.label} u0"""),
                        data = None,
                        short_desc = """Dummy L3 group added by Hao-Wei Pang""",
                        long_desc = f"""Dummy L3 group added by Hao-Wei Pang during fitting groups from thermo libraries""",
                        parent = node0)
        database_new.groups["group"].entries[group_atom.atomtype.label] = entry
        database_new.groups["group"].entries[node0.label].children.append(entry)
        node0 = database_new.groups["group"].entries[group_atom.atomtype.label]
        
    replace = False
    reason = ''
    if node0.data is None:
        reason = "None"
    elif isinstance(node0.data,str):
        reason = node0.data
    elif node0.data.is_all_zeros():
        reason = "all zeros"
        
    if reason:
        #if node0 data is missing, and the new group is the same as the parent group, 
        #then we replace the node0 data with the group data
        if node0.item.is_isomorphic(group):
            
            for atom in node0.item.atoms:
                if atom.label:
                    neighbors = ''.join(sorted([atom2.atomtype[0].label for atom2 in atom.edges.keys()
                                                if atom2.atomtype[0].label != 'H']))
                    neighbors += 'H' * len(['H' for atom2 in atom.edges.keys() if atom2.atomtype[0].label == 'H'])

                    node0_group_str = atom.atomtype[0].label

                    if neighbors:
                        node0_group_str += f'-{neighbors}'

                    if node0_group_str == entry_label:
                        replace = True
        
    missing_group_dict[group_str]["node0"] = node0
    
    if replace:
        #replace existing group entry
        database_new.groups['group'].entries[node0.label].data = fitted_group_data[group_str]
        database_new.groups['group'].entries[node0.label].short_desc = """Fitted from RMG thermo libraries by Hao-Wei Pang"""
        database_new.groups['group'].entries[node0.label].long_desc = f"""Previously contains {reason} as data. New data fitted from following species using Ridge regression

{string}

"""

        missing_group_dict[group_str]["entry"] = database_new.groups['group'].entries[node0.label]
        missing_group_dict[group_str]["replaced"] = True
    else:
        #make new group entry
        entry = Entry(index = len(database_new.groups['group'].entries)+1,
                        label = entry_label,
                        item = group,
                        data = fitted_group_data[group_str],
                        short_desc = """Group added & fitted from RMG thermo libraries by Hao-Wei Pang""",
                        long_desc = f"""Fitted from following species using Ridge regression

{string}

    """,
                         parent = node0)
        
        missing_group_dict[group_str]["entry"] = entry
        missing_group_dict[group_str]["replaced"] = False
        database_new.groups['group'].entries[entry_label] = entry
        database_new.groups["group"].entries[node0.label].children.append(entry)
        
        ################################ test #######################################
        #test that we can make sample molecule
        #test that the sample molecuke descend to itself or its ancestors
        #test that node0 is an appropriate parent for its child entry
        try:
            sample_molecule = entry.item.make_sample_molecule()
        except:
            logging.error("Problem making sample molecule for group {}\n{}".format(
                entryName, entry.item.to_adjacency_list()))
            raise
        # for now ignore sample atoms that use nitrogen types
        nitrogen = False
        for atm in sample_molecule.atoms:
            if atm.is_nitrogen():
                nitrogen = True

        if not nitrogen:
            atoms = sample_molecule.get_all_labeled_atoms()
            match = database_new.groups["group"].descend_tree(sample_molecule, atoms, strict=True)

            if entry not in [match] + database_new.groups["group"].ancestors(match):
                raise
                
        if not database_new.groups["group"].match_node_to_child(node0, entry):
            raise

# 5. Check that we land on the right node when using the new database

## 5.1 Estimate thermo using the new database

In [ ]:
all_data["New GAE"] = list()
for entry in all_data["entry"]:
    molecule = entry.item
    all_data["New GAE"].append(database_new.estimate_thermo_via_group_additivity(molecule))

## 5.2 Parity plot for the library entry vs. new estimation

In [ ]:
colormap = mpl.cm.tab20
plt.figure(figsize=(10,10))
entry_values = list()
old_GAE_values = list()
new_GAE_values = list()
new_GAE_uncertainties = list()
new_new_GAE_values = list()

for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_values.append(entry_thermo.H298.value_si/4180)
    estimated_GAE = all_data["GAE with estimated GAV"][spc_index]
    old_GAE_values.append(estimated_GAE.H298.value_si/4180)
    fitted_GAE = all_data["GAE with fitted GAV"][spc_index]
    new_GAE_values.append(fitted_GAE.H298.value_si/4180)
    new_GAE_uncertainties.append(fitted_GAE.H298.uncertainty_si/4180)
    new_new_GAE_values.append(all_data["New GAE"][spc_index].H298.value_si/4180)
    
    
plt.figure(figsize=(10,10))
plt.errorbar(entry_values,old_GAE_values,fmt='o')
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
RMSE = mean_squared_error(entry_values,old_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,old_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} kcal/mol')
plt.xlabel("H298: Entry data (kcal/mol)")
plt.ylabel("H298: Old estimation (kcal/mol)")

plt.figure(figsize=(10,10))
plt.errorbar(x=entry_values,y=new_GAE_values,yerr=new_GAE_uncertainties,fmt='o')
for i, entry_value in enumerate(entry_values):
    if abs(entry_value - new_GAE_values[i])>15:
        plt.text(entry_value, new_GAE_values[i], s=all_data["entry"][i].label)
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
plt.xlabel("H298: Entry data (kcal/mol)")
plt.ylabel("H298: New estimation with fitted data (kcal/mol)")
RMSE = mean_squared_error(entry_values,new_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,new_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} kcal/mol')

In [ ]:
colormap = mpl.cm.tab20
plt.figure(figsize=(10,10))
entry_values = list()
old_GAE_values = list()
new_GAE_values = list()
new_GAE_uncertainties = list()
new_new_GAE_values = list()


for spc_index, entry_thermo in enumerate(all_data["entry thermo"]):
    entry_values.append(entry_thermo.S298.value_si/4.180)
    estimated_GAE = all_data["GAE with estimated GAV"][spc_index]
    old_GAE_values.append(estimated_GAE.S298.value_si/4.180)
    fitted_GAE = all_data["GAE with fitted GAV"][spc_index]
    new_GAE_values.append(fitted_GAE.S298.value_si/4.180)
    new_GAE_uncertainties.append(fitted_GAE.S298.uncertainty_si/4.180)
    new_new_GAE_values.append(all_data["New GAE"][spc_index].S298.value_si/4.180)

plt.figure(figsize=(10,10))
plt.errorbar(entry_values,old_GAE_values,fmt='o')
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
RMSE = mean_squared_error(entry_values,old_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,old_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} cal/(mol*K)')
plt.xlabel("S298: Entry data (cal/(mol*K))")
plt.ylabel("S298: Old estimation (cal/(mol*K))")

plt.figure(figsize=(10,10))
plt.errorbar(x=entry_values,y=new_GAE_values,yerr=new_GAE_uncertainties,fmt='o')
for i, entry_value in enumerate(entry_values):
    if abs(entry_value - new_GAE_values[i])>3:
        plt.text(entry_value, new_GAE_values[i], s=all_data["entry"][i].label)
plt.plot([min(entry_values),max(entry_values)],[min(entry_values),max(entry_values)],'-', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)+5,max(entry_values)+5],'--', color=colormap(1))
plt.plot([min(entry_values),max(entry_values)],[min(entry_values)-5,max(entry_values)-5],'--', color=colormap(1))
plt.xlabel("S298: Entry data (cal/(mol*K))")
plt.ylabel("S298: New estimation with fitted data (cal/(mol*K))")
RMSE = mean_squared_error(entry_values,new_GAE_values,squared=False)
MAE = mean_absolute_error(entry_values,new_GAE_values)
plt.title(f'RMSE: {RMSE:.1f}, MAE: {MAE:.1f} cal/(mol*K)')

# 6. Make sure we pass database test

In [ ]:
group_name = "group"
group = database.groups["group"]

entries_copy = copy(group.entries)
tst = []
for node_name, node_group in group.entries.items():
    del entries_copy[node_name]
    for node_name_other, node_group_other in entries_copy.items():
        group.match_node_to_node(node_group, node_group_other)
        tst.append((group.match_node_to_node(node_group, node_group_other),
                    "Node {node} in {group} group was found to be identical to node {node_other}".format(
                        node=node_name, group=group_name, node_other=node_name_other)))

boo = False
for i in range(len(tst)):
    if tst[i][0]:
        logging.error(tst[i][1])
        boo = True

if boo:
    raise ValueError("Error Occurred")
    
    

In [ ]:
tst1 = []
tst2 = []
for node_name, child_node in group.entries.items():
    # top nodes and product nodes don't have parents by definition, so they get an automatic pass:
    if child_node in group.top:
        continue
    parent_node = child_node.parent
    # Check whether the node has proper parents unless it is the top reactant or product node
    # The parent should be more general than the child
    tst1.append((group.match_node_to_child(parent_node, child_node),
                 "In {group} group, node {parent} is not a proper parent of its child {child}.".format(
                     group=group_name, parent=parent_node, child=node_name)))

    # check that parentNodes which are LogicOr do not have an ancestor that is a Group
    # If it does, then the child_node must also be a child of the ancestor
    if isinstance(parent_node.item, LogicOr):
        ancestor_node = parent_node
        while ancestor_node not in group.top and isinstance(ancestor_node.item, LogicOr):
            ancestor_node = ancestor_node.parent
        if isinstance(ancestor_node.item, Group) and tst1[-1][0]:
            tst2.append((group.match_node_to_child(ancestor_node, child_node),
                         "In {group} group, node {ancestor} is not a proper ancestor of its child {child}."
                         "".format(group=group_name, ancestor=ancestor_node, child=node_name)))

boo = False
for i in range(len(tst1)):
    if not tst1[i][0]:
        logging.error(tst1[i][1])
        boo = True
for i in range(len(tst2)):
    if not tst2[i][0]:
        logging.error(tst2[i][1])
        boo = True

if boo:
    raise ValueError("Error Occurred")

# Save new database to file

In [ ]:
database_new.groups["group"].save(os.path.join(settings["database.directory"],"thermo/groups/group.py"))